## KITTI 데이터셋에서 Occlusion 관련해서 필요한 라벨들만 추출하고 YOLOv5 포맷으로 변경하기
- Occluded `0`인 온전한 자동차 객체의 bbox 라벨을 여러가지 버전으로 분할해서 라벨에 추가했을 때 학습 모델이 어떠한 효과를 내는지 시도하려 했음
- 실험은 진행하지 않았지만, 라벨 생성 과정을 아래에 기록

### 기본 경로 가져오기

In [34]:
import os

os.chdir('/Users/Jinmi/Desktop')
original_dir = '/Users/Jinmi/Desktop/label_2'
original_dir_list = os.listdir(original_dir)
original_dir_list = sorted(original_dir_list)

In [35]:
original_dir_list[:20]

['000000.txt',
 '000001.txt',
 '000002.txt',
 '000003.txt',
 '000004.txt',
 '000005.txt',
 '000006.txt',
 '000007.txt',
 '000008.txt',
 '000009.txt',
 '000010.txt',
 '000011.txt',
 '000012.txt',
 '000013.txt',
 '000014.txt',
 '000015.txt',
 '000016.txt',
 '000017.txt',
 '000018.txt',
 '000019.txt']

In [36]:
os.getcwd()

'/Users/Jinmi/Desktop'

### label -> Set 0번 포맷
- DontCare 삭제
- 'Car', 'Truck', 'Van' 세 개의 클래스에서 `Occluded '0'`인 것만 남기기(`Occluded '1'` 이상은 삭제)

In [37]:
# 새로 만들 라벨의 폴더 이름
folder_name = 'label_set0'
if not os.path.exists(folder_name):
    os.mkdir(folder_name)

desktop = '/Users/Jinmi/Desktop'
os.chdir(os.path.join(desktop, folder_name))
os.getcwd()

'/Users/Jinmi/Desktop/label_set0'

In [38]:
# 원본 라벨 확인
with open(os.path.join(original_dir, original_dir_list[2000]), 'r') as f:
    print(f.read())

Car 0.96 0 -0.86 0.00 201.43 303.98 369.00 1.50 1.78 3.69 -3.16 1.68 3.35 -1.56
Cyclist 0.00 1 -1.60 991.71 147.35 1029.66 217.34 1.72 0.78 1.71 10.47 0.90 18.34 -1.08
Car 0.00 1 -0.48 956.95 156.33 1124.69 205.06 1.51 1.75 4.45 14.38 0.76 23.55 0.06
Car 0.00 0 1.50 439.37 172.17 476.11 200.79 1.44 1.64 3.68 -7.94 1.02 37.94 1.29
Misc 0.00 0 1.76 370.74 169.93 394.17 203.93 1.87 0.86 2.12 -12.61 1.29 40.13 1.46
Car 0.00 1 1.92 224.79 176.21 310.48 218.48 1.63 1.78 4.54 -14.14 1.46 29.92 1.49
DontCare -1 -1 -10 874.10 153.15 911.13 194.29 -1 -1 -1 -1000 -1000 -1000 -10
DontCare -1 -1 -10 621.70 163.74 632.35 175.85 -1 -1 -1 -1000 -1000 -1000 -10



In [39]:
print(original_dir_list[:10])
print(original_dir_list[990:1000])

['000000.txt', '000001.txt', '000002.txt', '000003.txt', '000004.txt', '000005.txt', '000006.txt', '000007.txt', '000008.txt', '000009.txt']
['000990.txt', '000991.txt', '000992.txt', '000993.txt', '000994.txt', '000995.txt', '000996.txt', '000997.txt', '000998.txt', '000999.txt']


In [40]:
%%time
to_path = os.path.join(desktop, folder_name)
for txt in original_dir_list[1000:6000]: # 학습 시간을 고려해 5000개만 사용한 데이터셋을 생성할 예정
    
    # 기존 라벨 열기
    with open(os.path.join(original_dir, txt), 'r') as fr:
        lines = []
        for line in fr:
            word_list = line.split()
            
            # DontCare 제외
            if word_list[0] == 'DontCare':
                continue
                
            # 세 개의 클래스 중 하나면서 Occluded 0 아니면 제외하기    
            if (word_list[2] != '0') and ((word_list[0] == 'Car') or (word_list[0] == 'Van') or (word_list[0] == 'Truck')):
                continue
            
            # 나머지면 추가
            lines.append(line)
        with open(os.path.join(to_path, txt), 'w') as fw:
            fw.writelines(lines)       

CPU times: user 689 ms, sys: 1.35 s, total: 2.04 s
Wall time: 3.39 s


In [41]:
# 변경된 라벨 확인
with open(os.path.join(to_path, original_dir_list[2000]), 'r') as f:
    print(f.read())

Car 0.96 0 -0.86 0.00 201.43 303.98 369.00 1.50 1.78 3.69 -3.16 1.68 3.35 -1.56
Cyclist 0.00 1 -1.60 991.71 147.35 1029.66 217.34 1.72 0.78 1.71 10.47 0.90 18.34 -1.08
Car 0.00 0 1.50 439.37 172.17 476.11 200.79 1.44 1.64 3.68 -7.94 1.02 37.94 1.29
Misc 0.00 0 1.76 370.74 169.93 394.17 203.93 1.87 0.86 2.12 -12.61 1.29 40.13 1.46



### YOLOv5 라벨로 바꿔주기

In [42]:
try_path = '/Users/Jinmi/Desktop/yolo_labels_set0'
if not os.path.exists(try_path):
    os.mkdir(try_path)

In [44]:
import cv2
import pandas as pd

# 클래스 값을 숫자로 변환해주는 매핑 함수
def label_mapping(string):
    map_dict = {'Car' : 0,
                'Pedestrian' : 1,
                'Truck' : 2,
                'Cyclist' : 3,
                'Van' : 4,
                'Tram' : 5,
                'Misc' : 6,
                'Person_sitting' : 7,
                'DontCare' : 8,
                }
    return map_dict[string]

raw_label_root_path = '/Users/Jinmi/Desktop/label_set0'
raw_img_root_path = '/Users/Jinmi/Desktop/images/training/image_2'
label_filenames = os.listdir(raw_label_root_path)
to_path = try_path

# 라벨 새로 저장할 경로 생성
if not os.path.exists(to_path):
    os.mkdir(to_path)

# 변환 시 직관적으로 알 수 있게끔 해줄 판다스 데이터프레임 헤더 리스트
columns = ['type', 'truncated', 'occluded', 'alpha', 'bbox_l', 'bbox_t', 'bbox_r', 'bbox_b', '3d_h', '3d_w', '3d_l', '3d_x', '3d_y', '3d_z', '3d_rot_y', 'score']

# 이미지 경로 전체 돌면서 매칭되는 기존 라벨값 불러온 후 변환 및 다른 경로에 저장
for filename in label_filenames:
    lab_path = os.path.join(raw_label_root_path, filename)
    img_path = os.path.join(raw_img_root_path, filename[:6]+'.png')
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    height = img.shape[0]
    width = img.shape[1]
    img_id = img_path[-10:-4] # 이미지 넘버
    df = pd.read_csv(lab_path, sep=' ', names=columns)

    df['type'] = df['type'].map(label_mapping)
    df = df[df['type'] != 8] # 혹시 모를 DontCare 제외
    # df = df[(df['occluded'] == 1) | (df['occluded'] == 2)]
    df['ob_width'] = df['bbox_r'] - df['bbox_l']
    df['ob_height'] = df['bbox_b'] - df['bbox_t']
    df['x_center'] = df['ob_width'] / 2 + df['bbox_l']
    df['y_center'] = df['ob_height'] / 2 + df['bbox_t']
    df['norm_x_center'] = df['x_center'] / width
    df['norm_y_center'] = df['y_center'] / height
    df['norm_ob_width'] = df['ob_width'] / width
    df['norm_ob_height'] = df['ob_height'] / height

    df_to_save = df.loc[:, ['type', 'norm_x_center', 'norm_y_center', 'norm_ob_width', 'norm_ob_height']]
    df_to_save.to_csv(os.path.join(to_path, img_id + '.txt'), sep=' ', index=False, header=False)

In [45]:
# 개수 확인
len(os.listdir(try_path))

5000

In [49]:
# 예상된 것들이 잘 갔는지 확인
print(sorted(os.listdir(try_path))[:10])
print(sorted(os.listdir(try_path))[-10:])

['001000.txt', '001001.txt', '001002.txt', '001003.txt', '001004.txt', '001005.txt', '001006.txt', '001007.txt', '001008.txt', '001009.txt']
['005990.txt', '005991.txt', '005992.txt', '005993.txt', '005994.txt', '005995.txt', '005996.txt', '005997.txt', '005998.txt', '005999.txt']


In [54]:
original_dir_list[2000]

'002000.txt'

In [55]:
# YOLOv5 포맷 변경된 거 확인
with open(os.path.join(try_path, '002000.txt'), 'r') as f:
    print(f.read())

0 0.1241748366013072 0.7708513513513514 0.2483496732026144 0.4528918918918919
3 0.8257230392156863 0.4928243243243243 0.03100490196078435 0.1891621621621622
0 0.3739705882352941 0.504 0.030016339869281053 0.07735135135135136
6 0.3124632352941177 0.5052162162162163 0.019142156862745105 0.0918918918918919



### label -> Set 1번 포맷
- DontCare 삭제
- 'Car', 'Truck', 'Van' 세 개의 클래스에서 `Occluded '0'`인 것 남기기(`Occluded '1'` 이상은 삭제)
- 'Car', 'Truck', 'Van' 세 개의 클래스에서 `Occluded '0'`인 건 horizontally, vertically 50%인 것 총 4개 추가하기

In [56]:
desktop = '/Users/Jinmi/Desktop'
os.chdir(desktop)

folder_name = 'label_set1'
if not os.path.exists(folder_name):
    os.mkdir(folder_name)

os.chdir(os.path.join(desktop, folder_name))
os.getcwd()

'/Users/Jinmi/Desktop/label_set1'

In [57]:
%%time
to_path = os.path.join(desktop, folder_name)
for txt in original_dir_list[1000:6000]:
    with open(os.path.join(original_dir, txt), 'r') as fr:
        lines = []
        for line in fr:
            word_list = line.split()
            
            # DontCare 제외
            if word_list[0] == 'DontCare':
                continue
                
            # 세 개의 클래스 중 하나면서 Occluded 0 아니면 제외하기    
            if (word_list[2] != '0') and ((word_list[0] == 'Car') or (word_list[0] == 'Van') or (word_list[0] == 'Truck')):
                continue
            
            # 나머지면 추가
            lines.append(line)
            
            # Occlusion 0인 'Car', 'Van', 'Truck'은 bbox 4개 라벨 추가하기
            if (word_list[2] == '0') and ((word_list[0] == 'Car') or (word_list[0] == 'Van') or (word_list[0] == 'Truck')):
                half_x = str((float(word_list[6]) + float(word_list[4])) / 2)
                half_y = str((float(word_list[7]) + float(word_list[5])) / 2)
                additional_1 = [word_list[0], word_list[1], '4', word_list[3], word_list[4], word_list[5], half_x, word_list[7], 
                            word_list[8], word_list[9], word_list[10], word_list[11], word_list[12], word_list[13], word_list[14]]
                additional_2 = [word_list[0], word_list[1], '4', word_list[3], half_x, word_list[5], word_list[6], word_list[7], 
                            word_list[8], word_list[9], word_list[10], word_list[11], word_list[12], word_list[13], word_list[14]]
                additional_3 = [word_list[0], word_list[1], '4', word_list[3], word_list[4], half_y, word_list[6], word_list[7], 
                            word_list[8], word_list[9], word_list[10], word_list[11], word_list[12], word_list[13], word_list[14]]
                additional_4 = [word_list[0], word_list[1], '4', word_list[3], word_list[4], word_list[5], word_list[6], half_y, 
                            word_list[8], word_list[9], word_list[10], word_list[11], word_list[12], word_list[13], word_list[14]]  

                lines.append(' '.join(additional_1)+'\n')
                lines.append(' '.join(additional_2)+'\n')
                lines.append(' '.join(additional_3)+'\n')
                lines.append(' '.join(additional_4)+'\n')
            
        with open(os.path.join(to_path, txt), 'w') as fw:
            fw.writelines(lines)       

CPU times: user 949 ms, sys: 1.58 s, total: 2.53 s
Wall time: 3.84 s


In [58]:
# 변경된 라벨 확인
with open(os.path.join(to_path, original_dir_list[2000]), 'r') as f:
    print(f.read())

Car 0.96 0 -0.86 0.00 201.43 303.98 369.00 1.50 1.78 3.69 -3.16 1.68 3.35 -1.56
Car 0.96 4 -0.86 0.00 201.43 151.99 369.00 1.50 1.78 3.69 -3.16 1.68 3.35 -1.56
Car 0.96 4 -0.86 151.99 201.43 303.98 369.00 1.50 1.78 3.69 -3.16 1.68 3.35 -1.56
Car 0.96 4 -0.86 0.00 285.21500000000003 303.98 369.00 1.50 1.78 3.69 -3.16 1.68 3.35 -1.56
Car 0.96 4 -0.86 0.00 201.43 303.98 285.21500000000003 1.50 1.78 3.69 -3.16 1.68 3.35 -1.56
Cyclist 0.00 1 -1.60 991.71 147.35 1029.66 217.34 1.72 0.78 1.71 10.47 0.90 18.34 -1.08
Car 0.00 0 1.50 439.37 172.17 476.11 200.79 1.44 1.64 3.68 -7.94 1.02 37.94 1.29
Car 0.00 4 1.50 439.37 172.17 457.74 200.79 1.44 1.64 3.68 -7.94 1.02 37.94 1.29
Car 0.00 4 1.50 457.74 172.17 476.11 200.79 1.44 1.64 3.68 -7.94 1.02 37.94 1.29
Car 0.00 4 1.50 439.37 186.48 476.11 200.79 1.44 1.64 3.68 -7.94 1.02 37.94 1.29
Car 0.00 4 1.50 439.37 172.17 476.11 186.48 1.44 1.64 3.68 -7.94 1.02 37.94 1.29
Misc 0.00 0 1.76 370.74 169.93 394.17 203.93 1.87 0.86 2.12 -12.61 1.29 40.13 1.4

### YOLOv5 라벨로 바꿔주기

In [59]:
try_path = '/Users/Jinmi/Desktop/yolo_labels_set1'
if not os.path.exists(try_path):
    os.mkdir(try_path)

In [60]:
import cv2
import pandas as pd

# 클래스 값을 숫자로 변환해주는 매핑 함수
def label_mapping(string):
    map_dict = {'Car' : 0,
                'Pedestrian' : 1,
                'Truck' : 2,
                'Cyclist' : 3,
                'Van' : 4,
                'Tram' : 5,
                'Misc' : 6,
                'Person_sitting' : 7,
                'DontCare' : 8,
                }
    return map_dict[string]

raw_label_root_path = '/Users/Jinmi/Desktop/label_set1'
raw_img_root_path = '/Users/Jinmi/Desktop/images/training/image_2'
label_filenames = os.listdir(raw_label_root_path)
to_path = try_path

# 라벨 새로 저장할 경로 생성
if not os.path.exists(to_path):
    os.mkdir(to_path)

# 변환 시 직관적으로 알 수 있게끔 해줄 판다스 데이터프레임 헤더 리스트
columns = ['type', 'truncated', 'occluded', 'alpha', 'bbox_l', 'bbox_t', 'bbox_r', 'bbox_b', '3d_h', '3d_w', '3d_l', '3d_x', '3d_y', '3d_z', '3d_rot_y', 'score']

# 이미지 경로 전체 돌면서 매칭되는 기존 라벨값 불러온 후 변환 및 다른 경로에 저장
for filename in label_filenames:
    lab_path = os.path.join(raw_label_root_path, filename)
    img_path = os.path.join(raw_img_root_path, filename[:6]+'.png')
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    height = img.shape[0]
    width = img.shape[1]
    img_id = img_path[-10:-4] # 이미지 넘버
    df = pd.read_csv(lab_path, sep=' ', names=columns)

    df['type'] = df['type'].map(label_mapping)
    df = df[df['type'] != 8] # 혹시 모를 DontCare 제외
    # df = df[(df['occluded'] == 1) | (df['occluded'] == 2)]
    df['ob_width'] = df['bbox_r'] - df['bbox_l']
    df['ob_height'] = df['bbox_b'] - df['bbox_t']
    df['x_center'] = df['ob_width'] / 2 + df['bbox_l']
    df['y_center'] = df['ob_height'] / 2 + df['bbox_t']
    df['norm_x_center'] = df['x_center'] / width
    df['norm_y_center'] = df['y_center'] / height
    df['norm_ob_width'] = df['ob_width'] / width
    df['norm_ob_height'] = df['ob_height'] / height

    df_to_save = df.loc[:, ['type', 'norm_x_center', 'norm_y_center', 'norm_ob_width', 'norm_ob_height']]
    df_to_save.to_csv(os.path.join(to_path, img_id + '.txt'), sep=' ', index=False, header=False)

In [61]:
# YOLOv5 포맷 변경된 거 확인
with open(os.path.join(try_path, '002000.txt'), 'r') as f:
    print(f.read())

0 0.1241748366013072 0.7708513513513514 0.2483496732026144 0.4528918918918919
0 0.0620874183006536 0.7708513513513514 0.1241748366013072 0.4528918918918919
0 0.1862622549019608 0.7708513513513514 0.1241748366013072 0.4528918918918919
0 0.1241748366013072 0.8840743243243243 0.2483496732026144 0.22644594594594586
0 0.1241748366013072 0.6576283783783784 0.2483496732026144 0.22644594594594603
3 0.8257230392156863 0.4928243243243243 0.03100490196078435 0.1891621621621622
0 0.3739705882352941 0.504 0.030016339869281053 0.07735135135135136
0 0.36646650326797386 0.504 0.015008169934640526 0.07735135135135136
0 0.3814746732026144 0.504 0.015008169934640526 0.07735135135135136
0 0.3739705882352941 0.5233378378378378 0.030016339869281053 0.03867567567567568
0 0.3739705882352941 0.4846621621621621 0.030016339869281053 0.03867567567567568
6 0.3124632352941177 0.5052162162162163 0.019142156862745105 0.0918918918918919



### label -> Set 2번 포맷
- DontCare 삭제
- 'Car', 'Truck', 'Van' 세 개의 클래스에서 `Occluded '0', '1', '2', '3'` 전부 삭제
- 'Car', 'Truck', 'Van' 세 개의 클래스에서 `Occluded '0'`인 건 horizontally, vertically 50%인 것 총 4개 추가하기

In [62]:
desktop = '/Users/Jinmi/Desktop'
os.chdir(desktop)

folder_name = 'label_set2'
if not os.path.exists(folder_name):
    os.mkdir(folder_name)

os.chdir(os.path.join(desktop, folder_name))
os.getcwd()

'/Users/Jinmi/Desktop/label_set2'

In [63]:
%%time
to_path = os.path.join(desktop, folder_name)
for txt in original_dir_list[1000:6000]:
    with open(os.path.join(original_dir, txt), 'r') as fr:
        lines = []
        for line in fr:
            word_list = line.split()
            
            # DontCare 제외
            if word_list[0] == 'DontCare':
                continue

            # Occlusion 0인 'Car', 'Van', 'Truck'은 bbox 4개 라벨 추가하기
            if (word_list[2] == '0') and ((word_list[0] == 'Car') or (word_list[0] == 'Van') or (word_list[0] == 'Truck')):
                half_x = str((float(word_list[6]) + float(word_list[4])) / 2)
                half_y = str((float(word_list[7]) + float(word_list[5])) / 2)
                additional_1 = [word_list[0], word_list[1], '4', word_list[3], word_list[4], word_list[5], half_x, word_list[7], 
                            word_list[8], word_list[9], word_list[10], word_list[11], word_list[12], word_list[13], word_list[14]]
                additional_2 = [word_list[0], word_list[1], '4', word_list[3], half_x, word_list[5], word_list[6], word_list[7], 
                            word_list[8], word_list[9], word_list[10], word_list[11], word_list[12], word_list[13], word_list[14]]
                additional_3 = [word_list[0], word_list[1], '4', word_list[3], word_list[4], half_y, word_list[6], word_list[7], 
                            word_list[8], word_list[9], word_list[10], word_list[11], word_list[12], word_list[13], word_list[14]]
                additional_4 = [word_list[0], word_list[1], '4', word_list[3], word_list[4], word_list[5], word_list[6], half_y, 
                            word_list[8], word_list[9], word_list[10], word_list[11], word_list[12], word_list[13], word_list[14]]  

                lines.append(' '.join(additional_1)+'\n')
                lines.append(' '.join(additional_2)+'\n')
                lines.append(' '.join(additional_3)+'\n')
                lines.append(' '.join(additional_4)+'\n')

            # 세 개의 클래스 중 하나인 것들 원래 라벨은 전부 제외하기    
            if (word_list[0] == 'Car') or (word_list[0] == 'Van') or (word_list[0] == 'Truck'):
                continue
            
            # 나머지면 추가
            lines.append(line)
            
            
        with open(os.path.join(to_path, txt), 'w') as fw:
            fw.writelines(lines)       

CPU times: user 918 ms, sys: 1.53 s, total: 2.45 s
Wall time: 3.72 s


In [66]:
# 변경된 라벨 확인
with open(os.path.join(original_dir, original_dir_list[2000]), 'r') as f:
    print(f.read())

with open(os.path.join(to_path, original_dir_list[2000]), 'r') as f:
    print(f.read())
    
print()
print('==========')
print()

with open(os.path.join(original_dir, original_dir_list[3000]), 'r') as f:
    print(f.read())

with open(os.path.join(to_path, original_dir_list[3000]), 'r') as f:
    print(f.read())


Car 0.96 0 -0.86 0.00 201.43 303.98 369.00 1.50 1.78 3.69 -3.16 1.68 3.35 -1.56
Cyclist 0.00 1 -1.60 991.71 147.35 1029.66 217.34 1.72 0.78 1.71 10.47 0.90 18.34 -1.08
Car 0.00 1 -0.48 956.95 156.33 1124.69 205.06 1.51 1.75 4.45 14.38 0.76 23.55 0.06
Car 0.00 0 1.50 439.37 172.17 476.11 200.79 1.44 1.64 3.68 -7.94 1.02 37.94 1.29
Misc 0.00 0 1.76 370.74 169.93 394.17 203.93 1.87 0.86 2.12 -12.61 1.29 40.13 1.46
Car 0.00 1 1.92 224.79 176.21 310.48 218.48 1.63 1.78 4.54 -14.14 1.46 29.92 1.49
DontCare -1 -1 -10 874.10 153.15 911.13 194.29 -1 -1 -1 -1000 -1000 -1000 -10
DontCare -1 -1 -10 621.70 163.74 632.35 175.85 -1 -1 -1 -1000 -1000 -1000 -10

Car 0.96 4 -0.86 0.00 201.43 151.99 369.00 1.50 1.78 3.69 -3.16 1.68 3.35 -1.56
Car 0.96 4 -0.86 151.99 201.43 303.98 369.00 1.50 1.78 3.69 -3.16 1.68 3.35 -1.56
Car 0.96 4 -0.86 0.00 285.21500000000003 303.98 369.00 1.50 1.78 3.69 -3.16 1.68 3.35 -1.56
Car 0.96 4 -0.86 0.00 201.43 303.98 285.21500000000003 1.50 1.78 3.69 -3.16 1.68 3.35 -1.56


### YOLOv5 라벨로 바꿔주기

In [67]:
try_path = '/Users/Jinmi/Desktop/yolo_labels_set2'
if not os.path.exists(try_path):
    os.mkdir(try_path)

In [68]:
import cv2
import pandas as pd

# 클래스 값을 숫자로 변환해주는 매핑 함수
def label_mapping(string):
    map_dict = {'Car' : 0,
                'Pedestrian' : 1,
                'Truck' : 2,
                'Cyclist' : 3,
                'Van' : 4,
                'Tram' : 5,
                'Misc' : 6,
                'Person_sitting' : 7,
                'DontCare' : 8,
                }
    return map_dict[string]

raw_label_root_path = '/Users/Jinmi/Desktop/label_set2'
raw_img_root_path = '/Users/Jinmi/Desktop/images/training/image_2'
label_filenames = os.listdir(raw_label_root_path)
to_path = try_path

# 라벨 새로 저장할 경로 생성
if not os.path.exists(to_path):
    os.mkdir(to_path)

# 변환 시 직관적으로 알 수 있게끔 해줄 판다스 데이터프레임 헤더 리스트
columns = ['type', 'truncated', 'occluded', 'alpha', 'bbox_l', 'bbox_t', 'bbox_r', 'bbox_b', '3d_h', '3d_w', '3d_l', '3d_x', '3d_y', '3d_z', '3d_rot_y', 'score']

# 이미지 경로 전체 돌면서 매칭되는 기존 라벨값 불러온 후 변환 및 다른 경로에 저장
for filename in label_filenames:
    lab_path = os.path.join(raw_label_root_path, filename)
    img_path = os.path.join(raw_img_root_path, filename[:6]+'.png')
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    height = img.shape[0]
    width = img.shape[1]
    img_id = img_path[-10:-4] # 이미지 넘버
    df = pd.read_csv(lab_path, sep=' ', names=columns)

    df['type'] = df['type'].map(label_mapping)
#     df = df[df['type'] != 8] # 혹시 모를 DontCare 제외
    # df = df[(df['occluded'] == 1) | (df['occluded'] == 2)]
    df['ob_width'] = df['bbox_r'] - df['bbox_l']
    df['ob_height'] = df['bbox_b'] - df['bbox_t']
    df['x_center'] = df['ob_width'] / 2 + df['bbox_l']
    df['y_center'] = df['ob_height'] / 2 + df['bbox_t']
    df['norm_x_center'] = df['x_center'] / width
    df['norm_y_center'] = df['y_center'] / height
    df['norm_ob_width'] = df['ob_width'] / width
    df['norm_ob_height'] = df['ob_height'] / height

    df_to_save = df.loc[:, ['type', 'norm_x_center', 'norm_y_center', 'norm_ob_width', 'norm_ob_height']]
    df_to_save.to_csv(os.path.join(to_path, img_id + '.txt'), sep=' ', index=False, header=False)

In [69]:
# YOLOv5 포맷 변경된 거 확인
with open(os.path.join(try_path, '002000.txt'), 'r') as f:
    print(f.read())

0 0.0620874183006536 0.7708513513513514 0.1241748366013072 0.4528918918918919
0 0.1862622549019608 0.7708513513513514 0.1241748366013072 0.4528918918918919
0 0.1241748366013072 0.8840743243243243 0.2483496732026144 0.22644594594594586
0 0.1241748366013072 0.6576283783783784 0.2483496732026144 0.22644594594594603
3 0.8257230392156863 0.4928243243243243 0.03100490196078435 0.1891621621621622
0 0.36646650326797386 0.504 0.015008169934640526 0.07735135135135136
0 0.3814746732026144 0.504 0.015008169934640526 0.07735135135135136
0 0.3739705882352941 0.5233378378378378 0.030016339869281053 0.03867567567567568
0 0.3739705882352941 0.4846621621621621 0.030016339869281053 0.03867567567567568
6 0.3124632352941177 0.5052162162162163 0.019142156862745105 0.0918918918918919



### label -> Set 3번 포맷
- DontCare 삭제
- 'Car', 'Truck', 'Van' 세 개의 클래스에서 `Occluded '0', '1', '2', '3'` 전부 삭제
- 'Car', 'Truck', 'Van' 세 개의 클래스에서 `Occluded '0'`인 건 horizontally, vertically 25%인 것 총 4개 추가하기

In [70]:
desktop = '/Users/Jinmi/Desktop'
os.chdir(desktop)

folder_name = 'label_set3'
if not os.path.exists(folder_name):
    os.mkdir(folder_name)

os.chdir(os.path.join(desktop, folder_name))
os.getcwd()

'/Users/Jinmi/Desktop/label_set3'

_실험 중단으로 해당 라벨은 생성하지 않음_